In [10]:
import pygame # used this library in order to create game window and display.
import random # used this library in order to choose random words.
import sys # used this library in order to quit from the game.

pygame.init() # imported necessary modules before starting to use the pygame library.

# set the size of the game window and specified what should be written at the top of the game.

WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Welcome to Hangman Car Game")

# Colors
YELLOW = (255, 255, 0)
BLACK = (0, 0, 0)
NAVY_BLUE = (0, 0, 128)
RED = (255, 0, 0)

# Fonts
LETTER_FONT = pygame.font.SysFont('comicsans', 40)
WORD_FONT = pygame.font.SysFont('comicsans', 60)
TITLE_FONT = pygame.font.SysFont('comicsans', 50)
WIN_FONT = pygame.font.SysFont('comicsans', 80, bold=True)  # "You Win!" yazısı için yazı büyüklüğü ve kalın
LOSE_FONT = pygame.font.SysFont('comicsans', 80, bold=True)  # "You Lose!" yazısı için yazı büyüklüğü ve kalın
REPLAY_FONT = pygame.font.SysFont('comicsans', 40, bold=True)  # "Play Again?" yazısı için yazı büyüklüğü ve kalın
GUESS_LEFT_FONT = pygame.font.SysFont('comicsans', 25)  # "Guesses Left:" yazısı için yazı büyüklüğü

# called the images loaded into the directory on the left 
# the reason for having 7 different files is to show each step the character takes in case of a wrong guess. 
# obtained the relevant images from the game available at the following link: https://www.ebhasin.com/games/games-source/javascript/Hang-Man-(Bollywood-Movies)/src/index.html.
hangmanPics = [pygame.image.load('h0.png'), pygame.image.load('h1.png'), pygame.image.load('h2.png'),
               pygame.image.load('h3.png'), pygame.image.load('h4.png'), pygame.image.load('h5.png'),
               pygame.image.load('h6.png')]

# Words
CarBrandList = ["volkswagen", "toyota", "ford", "mercedes", "renault", "peugeot", "skoda",
                "nissan", "opel", "hyundai", "citroen", "mazda", "mini",
                "lexus", "jeep", "landrover", "dodge", "jaguar", "alfaromeo", "isuzu",
                "lotus", "bentley", "rollsroyce", "lamborghini", "casalini", "nordic",
                "mclaren", "alpine", "excalibur", "bugatti", "chevrolet", "ssangyong", "smart", "porsche", "lada"]

# used this function for drawing texts considering font, color and their coordination over the surface. 
# surface.blit function is used for drawing words and hangman.
def draw_text(text, font, color, surface, x, y, align="center"):
    text_obj = font.render(text, True, color)
    text_rect = text_obj.get_rect()
    if align == "center":
        text_rect.center = (x, y)
    elif align == "right":
        text_rect.topright = (x, y)
    elif align == "bottom_right":
        text_rect.bottomright = (x, y)
    elif align == "bottom_left":
        text_rect.bottomleft = (x, y)
    surface.blit(text_obj, text_rect)

# Function to draw hangman
def draw_hangman(screen, status):
    screen.blit(hangmanPics[status], (WIDTH // 2 - hangmanPics[status].get_width() // 2, 100))

# created this function to type the guessed letters.
# when player typed the letter that is not in the word, "" occurs on the screen which means nothing comes to screen. 
def draw_word(screen, word, guessed):
    display_word = ""
    for char in word:
        if char in guessed:
            display_word += char + " "
        else:
            display_word += "_ "
    draw_text(display_word, WORD_FONT, YELLOW, screen, WIDTH // 2, 400)

# Function to ask for replay
# used event.get function to listen the interaction with keyboard or mouse
# when player presses y button, game restarts (that is why return True). otherwise player quits the game
def ask_for_replay(screen):
    draw_text("Play Again? Press Y or N", REPLAY_FONT, RED, screen, WIDTH // 2, HEIGHT // 4, align="center")
    pygame.display.update()
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False           # oyunun tekrar oynanmayacağını gösterir.
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_y:
                    return True   # oyunun tekrar oynanacağını gösterir
                elif event.key == pygame.K_n:
                    return False

# this function is used to inform player regarding the guessed letter is not in the word.
def show_letter_not_in_word(screen):
    draw_text("This letter is not in the word.", REPLAY_FONT, RED, screen, WIDTH // 2, HEIGHT // 2)
    pygame.display.update()
    pygame.time.delay(1000) 
    
# this function is used to inform player regarding multiple usage for each letter. 
def show_letter_already_guessed(screen):
    draw_text("This letter has been already guessed.", REPLAY_FONT, RED, screen, WIDTH // 2, HEIGHT // 2)
    pygame.display.update()
    pygame.time.delay(1000)

# Main game loop
def main():
    while True:
        guessed_letters = set() 
        hangman_status = 0 # 
        max_guesses = 6
        word = random.choice(CarBrandList)
        correct_guesses = set(word)
        guessed_word = ""  # this place is assigned empty at the beginning.

        # Game loop
        # max_guesses - hangman status indicates the guess left value. For example, at the beginning this value 0 because there is no movement.
        run = True
        while run:
            screen.fill(YELLOW)
            draw_text("Welcome to Hangman Car Game", TITLE_FONT, BLACK, screen, WIDTH // 2, 50)
            draw_text(f"Guesses Left: {max_guesses - hangman_status}", GUESS_LEFT_FONT, BLACK, screen, WIDTH - 15,
                      HEIGHT - 15, align="bottom_right")  # Guess left is placed at the bottom right.
            draw_hangman(screen, hangman_status)
            draw_word(screen, word, guessed_letters)

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_ESCAPE:
                        pygame.quit()
                        sys.exit()
                    elif event.unicode.isalpha() and event.unicode.lower() not in guessed_letters: # unicodeis kısmı klavyede sadece girilen karakterlerin harf olup olmadığını izler
                        guessed_letters.add(event.unicode.lower())
                        if event.unicode.lower() in correct_guesses:
                            guessed_word += event.unicode.lower() 
                            if all(char in guessed_letters for char in correct_guesses):
                                draw_word(screen, word, guessed_letters)  # after the correct guess, the word pops up the screen.
                                pygame.display.update()
                                pygame.time.delay(2000) # you win pops up the screen after 2 seconds
                                draw_text("You Win!", WIN_FONT, NAVY_BLUE, screen, WIDTH // 2, HEIGHT // 2, align="center")
                                pygame.display.update()
                                pygame.time.delay(2000) 
                                run = False
                                break
                            continue
                        else:
                            show_letter_not_in_word(screen) 
                        hangman_status += 1
                    elif event.unicode.isalpha() and event.unicode.lower() in guessed_letters:
                        show_letter_already_guessed(screen) 

            if hangman_status >= max_guesses:
                draw_hangman(screen, hangman_status)
                pygame.display.update()
                pygame.time.delay(2000)  # wait 2 seconds
                draw_text("You Lose!", LOSE_FONT, RED, screen, WIDTH // 2, HEIGHT // 2, align="center")
                pygame.display.update()
                pygame.time.delay(3000)  # wait 2 seconds
                draw_text(f"The word was: {word.capitalize()}", TITLE_FONT, RED, screen, WIDTH // 2, HEIGHT // 2 + 100, align="center")
                pygame.display.update()
                pygame.time.delay(2000)  # wait 2 seconds
                break

            pygame.display.update()

        if not ask_for_replay(screen):  # if player does not want to play, in other words ask for replay return false, the loop ends.
            break

    pygame.quit()
    
if __name__ == "__main__":
    main()
